# Numerical Tic Tac Toe with minimax AI 
**WHO Academy coding challenge** <br/>
Alexis Navarian - 07/06/2020 <br/>

***Rules :***
- Turn by turn game on a n x n board
- Goal is to make a line, column or diagonal equal to $\frac{n(n^2+1)}{2}$.    $\quad$ Ex : n = 3 $\Rightarrow$ 15
- Player 1 can only use odd numbers while Player 2 can only use even numbers. $\quad$ Ex : n = 3 $\Rightarrow$ P1 : [1,3,5,7] - P2 : [2,4,6,8]

***Assumptions :***
- **One number can't be placed twice. Once a number is placed, it disappear from the list of possibilities.**
- A player wins only if a row/column/diagonal is FULL and its sum is equal to 15 (for n=3). $\quad$ Ex : (7 8 EmptyCell) doesn't work

***Coding idea :***
- Super class Player with attributes "name" and "numbers". Numbers is a list that represent the available numbers for the player. 
- Class Human_Player and AI_Player inherits from the Player class and have a specific function "get_move".
- Human_Player "get_move" function uses user input 
- AI_player "get_move" function uses the game configuration to output the best move calculated with an algorithm. $\quad$ Ex : minimax or random
- class Game : implementation of the whole game logic

***Project specificities :***
- should work for every board size (not only 3)
- class implementation 
- minimax with alpha beta pruning as AI algorithm
- AI algorithms and parameters comparison

***Warnings :***
- If AI is taking too much time to compute its move, please consider reducing the max_depth parameter (for minimax only)
- If you try to load a jupyter notebook cell while the program is waiting for your turn input, it will make the notebook "run forever/crash". If it happens, just close and restart the notebook. - If you want to stop the actual game early, enter 'STOP' in the input field.

In [1]:
import numpy as np
from IPython.display import clear_output  # Jupyter Notebook function to clear outputs

# Basic Player vs Player 

**class Player** <br/>
Player(name = 'Bob')

In [2]:
class Player:
    def __init__(self,name = 'Bob'):
        self.numbers = []  # list of available numbers to player
        self.name = name
    
    def set_numbers(self,numbers): # Not really sure that setters are useful in Python but anyway
        self.numbers = numbers
        
    def print_infos(self):  # Print info and available numbers list before playing a turn 
        print("Please, select a cell and a number. Answer format : column_letter row_number available_number ")
        print("Ex : 'A16' for column A, row 1 and number 6")
        print("Available numbers : ",self.numbers)
        
    def update_possibible_numbers(self,v):  # Update list - Once a number is used, it is removed from list
        self.numbers.remove(v)      

**class Human_Player** <br/>
Human_Player()

In [3]:
class Human_Player(Player):
    def get_move(self,board,mynumbers,opppnumbers):  # board and numbers aren't used here but are used for AI_Player.get_move 
        move = input()  
        return move

**class Game** <br/>
Game(n = 3, player0 = Human_Player(name = 'Bob'), player1 = Human_Player(name = 'John'), clear_output = True, verbose = True)

In [4]:
class Game:
    # Game initialization 
    def __init__(self, n=3, player0 = Human_Player(name = 'Bob'), player1 = Human_Player(name = 'John'), clear_output = True, verbose = True):  
        self.n = n 
        
        self.board = np.zeros((self.n,self.n),dtype = int)     # Create board of ize n x n
        self.win_limit = int(self.n*(self.n**2+1)/2)           # Define win limit. Ex: n = 3 => win_limit = 15
        
        self.player0 = player0                                 # Initialize players' lists
        self.player0.set_numbers(list(range(1,self.n**2+1,2))) # Player 0 gets odd numbers
        self.player1 = player1
        self.player1.set_numbers(list(range(2,self.n**2,2)))   # Player 1 gets even numbers 
        self.players = [self.player0,self.player1]
        
        self.turn = 0                                          # turn = 0 => player0's turn    turn = 1 => player1's turn
        self.state = 0                                         # Game state. 0 "continue" - 1 "Draw" - 2 "A player has won"
        
        self.winner = 'Draw'                                   # Winner is set if state = 2 - It is used for statistics 
        
        
        self.clear_output = clear_output                       # if set to True, it clears output at each game step
                                                               # if you want to debugg, set it to False
        self.verbose = verbose                                 # everything is visible only if verbose == true
                                                               # verbose = False is used for the statistical part
    
    # Function to print board 
    def print_board(self,):
        interligne = '  '+'|___'*self.n+"|"
        firstline = '  '+'____'*self.n+"_" 
        indexline = ' '+''.join(['   '+chr(x+65) for x in range(self.n)])  

        print(indexline)
        print(firstline)

        for ind,line in enumerate(self.board):
            s=str(ind+1)+" "
            for element in line:
                if element != 0:     
                    s = s +"| "+ str(element)+" "
                else:  
                    s= s +"|   "
            s = s+"|"
            print(s)
            print(interligne)        
    
    # Function to check if a move is correct, ie input only has 3 elements and all elements are valid
    def check_move(self,move):
        valid = True
        # check if len = 3 and row number is valid and column letter is valid 
        if len(move) != 3 or ord(move[0])<65 or ord(move[0])>=ord(chr(self.n+65)) or int(move[1])<1 or int(move[1])>self.n: 
            valid = False
        # check if board cell is free
        if  self.board[int(move[1])-1,ord(move[0])-65]!=0:
            valid = False
        # check if choosen numbers is available (not already used or not in list)
        if int(move[2]) not in self.players[self.turn].numbers:
            valid = False
        return valid
    
    # Convert 'A11' (Column_Letter Row_number choosen_number) format to 001 (row column value) format 
    def convert_move(self,move):
        r = int(move[1])-1
        c = ord(move[0])-65
        v = int(move[2])
        return (r,c,v)
    
    # update board and actual player's number list
    def process_move(self,r,c,v):
        self.board[r,c] = v
        self.players[self.turn].numbers.remove(v)
    
    # check if board state has changed after move represented by r,c,v - state = 0 : Continue, 1 : Draw and 2 : Victory
    def update_state(self,r,c,v):
        # Draw 
        if not np.any(self.board==0): #If no more free cell -> draw
            self.state = 1 

        # Horizontal win - sum line = 15 and no empty cell on line
        if sum(self.board[r,:])==self.win_limit and sum(self.board[r,:]==0)==0 : 
            self.state = 2

        # Vertical win - sum column = 15 and no empty cell on column
        if sum(self.board[:,c])==self.win_limit and sum(self.board[:,c]==0)==0 : 
            self.state = 2

        # Diagonals win - sum diag = 15 and no empty cell on diagonal
        # x==y => diag  
        if r==c:
            sum_diag = sum([self.board[i,i] for i in range(self.n)])
            empty_cell_diag = sum([self.board[i,i]==0 for i in range(self.n)])
            if sum_diag == self.win_limit and empty_cell_diag ==0:
                self.state = 2
        # x+y==n-1 => anti-diag - sum anti-diag = 15 and no empty cell on anti-diagonal
        elif r+c==self.n-1:
            sum_anti_diag = sum([self.board[self.n-i-1,i] for i in range(self.n)])
            empty_cell_anti_diag = sum([self.board[self.n-i-1,i]==0 for i in range(self.n)])
            if sum_anti_diag == self.win_limit and empty_cell_anti_diag ==0:
                self.state = 2
    
    def get_winner(self):
        return self.winner
    
    # game loop
    def play(self):
        #While state = "continue"
        while self.state == 0:
            if self.clear_output: clear_output() # Jupyter Notebook function to clear output (for better visualization)
            
            # print player available numbers and actual board
            if self.verbose :
                self.players[self.turn].print_infos()
                self.print_board()
            
            # get player's move
            move = self.players[self.turn].get_move(self.board,self.players[self.turn].numbers,self.players[1-self.turn].numbers)
            
            # check if move is a valid one
            is_proper_move = self.check_move(move)
            
            # if not, ask again for a valid move
            if not is_proper_move:
                continue
            
            # else, convert move to row column value format 
            (r,c,v) = self.convert_move(move)
            
            # update game with move 
            self.process_move(r,c,v)
            
            # check if game state has changed 
            self.update_state(r,c,v)
            
            # if state has changed, handle end game
            if self.state == 1 :
                if self.clear_output: clear_output()
                if self.verbose : 
                    self.print_board()
                    print("No more possible moves ! This is a draw")
                break 
            elif self.state == 2 : 
                if self.clear_output: clear_output()
                if self.verbose :
                    self.print_board()
                    print("Victory ! Player : "+self.players[self.turn].name)
                self.winner = self.players[self.turn].name
                break
            
            #else, turn to other player
            self.turn = 1 - self.turn
            
            
    

**Player vs Player game** <br/>
Game(n)

*Main points* : 
- Game class is by default using two Human_Players, printing everything and clearing it after each game step. 
- To modify printing and clearing options, please look at verbose and clear_output attributes of game class.
- Player0 uses odd numbers and Player1 uses even numbers 
- AI mode is presented in the next section 
- Game has been tested with n = 3 but feel free to try it with other n values (everything should work properly).

*Input format* :
- At each turn please enter an input of the following format (other input types won't be accepted) : <br/>

 **abc** <br/>
 Where : <br/>
     a is the column letter in UPPER CASE <br/>
     b is the row number starting at 1 <br/>
     c is the chosen number which is a number available to the player<br/>
     NB : available numbers are given before each input step

*Example :* " My available number list is [1,3,5,9] and I want to put a 9 in cell A1, then I type A19 without brackets and spaces between characters".
     

In [5]:
player_vs_player_game = Game(3,player0 = Human_Player(name = 'Myself-1'),player1 = Human_Player(name = 'Myself-2'))
player_vs_player_game.play()

    A   B   C
  _____________
1 |   |   | 8 |
  |___|___|___|
2 | 1 | 2 |   |
  |___|___|___|
3 | 5 |   |   |
  |___|___|___|
Victory ! Player : Myself-2


# AI mode

**class AI_Player**<br/>
AI_Player(max_depth = 9, algo = 'minimax', debugging = False, verbose = True)

*Supported algorithms :* 
- 'minimax' : minimax algorithm with alpha beta pruning - Explanations and Pseudo code here : https://www.youtube.com/watch?v=l-hh51ncgDI
- 'random' : random AI that do random (but valid) moves.

In [6]:
class AI_Player(Player):  
    # Initialization 
    def __init__(self,name,max_depth=9,algo='minimax',debugging = False, verbose = True):
        
        Player.__init__(self,name)                                       # inherits super class attributes 
        
        self.max_depth = max_depth                                       # set max_depth for minimax algo
        self.algo = algo                                                 # set algo used to compute best move 
         
        self.debugging = debugging                                       # if set to True, print debugging info
        self.verbose = verbose                                           # if set to True, print when AI is thinking
    
    # Name is pretty explicit - cell 'A11' -> 001
    def convert_rcv_to_input_format(self,r,c,v):
        return chr(65+c)+str(r+1)+str(v)
    
    # Check game state (Continue - 0 ,Draw - 1, Win - 2)
    def get_game_state(self,board):
        n = board.shape[0]
        win_limit = int(n*(n**2+1)/2)
        
        lines_sum = np.sum(board,axis = 1)
        nb_empty_cells_lines = np.sum(board==0,axis = 1)
        
        columns_sum = np.sum(board,axis = 0)
        nb_empty_cells_columns = np.sum(board==0,axis = 0)
        
        diag_sum = np.sum(np.diag(board))
        nb_empty_cells_diag = np.sum(np.diag(board==0))
        
        anti_diag_sum = np.sum(np.diag(np.fliplr(board)))
        nb_empty_cells_anti_diag = np.sum(np.diag(np.fliplr(board==0)))
        
        still_empty_cells = np.any(board==0)
        
        state = 0
        if not still_empty_cells: 
            state = 1
        for i in range(n):
            if lines_sum[i]==win_limit and nb_empty_cells_lines[i] == 0:
                state = 2
                break
            if columns_sum[i]==win_limit and nb_empty_cells_columns[i] == 0:
                state = 2
                break
        if diag_sum == win_limit and nb_empty_cells_diag ==0:
            state = 2
        if anti_diag_sum == win_limit and nb_empty_cells_anti_diag == 0 :
            state = 2
        return state 
    
    # Create hallucinated next board for minimax algo 
    def get_child(self,board,r,c,v):
        board_child = board.copy()
        board_child[r,c] = v
        return board_child
    
    # minimax algorithm with alpha-beta pruning 
    # Really good explanations and pseudo-code can be found here: https://www.youtube.com/watch?v=l-hh51ncgDI
    # Score explanation : score is 0 if Draw, -1 if lose and +1 if win 
    # We add + or - 'depth/max_depth' to this score to make it chose the "shortest" path leading to the best outcome
    def minimax(self,board,depth,alpha,beta,maximizingPlayer,mynumbers,oppnumbers): # recursive function
        n = board.shape[0]
        best_r = None; best_c = None; best_v = None
        
        # Leaf case
        state = self.get_game_state(board)
        if depth == 0 or state >0:
            return (state+depth/self.max_depth-1)*(1-maximizingPlayer)+(1-state-depth/self.max_depth)*maximizingPlayer,best_r,best_c,best_v
        
        # Else continue until Leaf
        if maximizingPlayer:
            maxEval = -10
            for r in range(n):
                for c in range(n):
                    if board[r,c] != 0: # if board cell isn't free, continue to next cell
                        continue
                    for v in mynumbers:
                        board_child = self.get_child(board,r,c,v)
                        mynumbers_c = mynumbers.copy(); mynumbers_c.remove(v)
                        score,_,_,_ = self.minimax(board_child,depth-1,alpha,beta,False,mynumbers_c,oppnumbers)
                        if score > maxEval:
                            best_r = r
                            best_c = c
                            best_v = v
                            maxEval = score
                        alpha = max(alpha,score)
                        if beta <= alpha:
                            return maxEval,best_r,best_c,best_v
                        if depth == self.max_depth and self.debugging: 
                            print(r,c,v,score,maxEval,alpha,beta)
            return maxEval,best_r,best_c,best_v
        else:
            minEval = 10
            for r in range(n):
                for c in range(n):
                    if board[r,c] != 0: # if board cell isn't free, continue to next cell
                        continue
                    for v in oppnumbers:
                        board_child = self.get_child(board,r,c,v)
                        oppnumbers_c = oppnumbers.copy(); oppnumbers_c.remove(v)
                        score,_,_,_ = self.minimax(board_child,depth-1,alpha,beta,True,mynumbers,oppnumbers_c)
                        if score < minEval:
                            best_r = r
                            best_c = c
                            best_v = v
                            minEval = score
                        beta = min(beta,score)
                        if beta <= alpha:
                            return minEval,best_r,best_c,best_v
            return minEval,best_r,best_c,best_v
        
    # AI using random moves    
    def get_random_move(self,board):
        valid_rows, valid_columns = np.where(board==0)
        random_ind = np.random.randint(low = 0, high = len(valid_rows))
        random_row = valid_rows[random_ind]
        random_column = valid_columns[random_ind]
        random_value = self.numbers[np.random.randint(low = 0,high = len(self.numbers))]
        return random_row,random_column,random_value
        
        
    # Get AI's best move using desired algorithm + convert to right format
    def get_move(self,board,mynumbers,oppnumbers):
        if self.verbose:
            print("AI player is thinking. Please wait a second.")
            print("If AI is taking too much time, please consider reducing the max_depth parameter")
        if self.algo == 'minimax':
            s,r,c,v = self.minimax(board,self.max_depth,-10000,10000,True,mynumbers,oppnumbers)
            # Security measure in case no optimal solution is found
            if r == None:
                r,c,v = self.get_random_move(board)
            move = self.convert_rcv_to_input_format(r,c,v)
        elif self.algo == 'random':
            r,c,v = self.get_random_move(board)
            move = self.convert_rcv_to_input_format(r,c,v)
        else:
            print("Algo not definied")
        return move

**Player vs AI game** <br/>
Game(n,player0 = Human_Player(name = 'Myself'), player1 = AI_Player(name = 'AlexNvn minimax 1.0',max_depth=9,algo='minimax'))

*Main points :*
- To modify printing and clearing options, please look at verbose and clear_output attributes of game class.
- Player0 uses odd numbers and Player1 uses even numbers. If you want AI to use odd numbers, declare player0 as AI_player and player1 as Human_Player.
- AI is by default using minimax algorithm with max_depth = 9. 
- **For n=3 and max_depth = 9, it took ~30sec on my laptop to compute first move (next moves are faster).**
- If AI is taking too much time to compute its move, please consider reducing the max_depth parameter.
- Game has been tested with n = 3 but feel free to test it with other n values (everything should work properly but AI might require a lot of time to compute its move).

*Input format :*
- At each turn please enter an input of the following format (other input types won't be accepted) : <br/>

*Available AI algorithms :*
- 'minimax' : minimax algorithm with alpha beta pruning - Explanations and Pseudo code here : https://www.youtube.com/watch?v=l-hh51ncgDI
- 'random' : random AI that do random (but valid) moves.

 **abc** <br/>
 Where : <br/>
     a is the column letter in UPPER CASE <br/>
     b is the row number starting at 1 <br/>
     c is the chosen number which is a number available to the player<br/>
     NB : available numbers are given before each input step

*Example :* " My available number list is [1,3,5,9] and I want to put a 9 in cell A1, then I type A19 without brackets and spaces between characters".

**Versus minimax AI**

In [7]:
player_vs_minimax_game = Game(3,player0 = Human_Player(name = 'Myself'), player1 = AI_Player(name = 'AlexNvn minimax 1.0',max_depth=9,algo='minimax'))
player_vs_minimax_game.play()

    A   B   C
  _____________
1 | 2 |   | 3 |
  |___|___|___|
2 | 8 |   |   |
  |___|___|___|
3 | 5 |   |   |
  |___|___|___|
Victory ! Player : AlexNvn minimax 1.0


**Versus random AI** 

In [75]:
player_vs_minimax_game = Game(3,player0 = Human_Player(name = 'Myself'), player1 = AI_Player(name = 'Random AI',algo='random'))
player_vs_minimax_game.play()

    A   B   C
  _____________
1 | 3 | 1 | 9 |
  |___|___|___|
2 | 7 | 5 | 6 |
  |___|___|___|
3 | 4 | 8 | 2 |
  |___|___|___|
No more possible moves ! This is a draw


# AI vs AI Results

*What have been done*
- Comparison of AI models.
- Results are obtained over 1 game (since everything is completely determined except what the random AI does) on the 3x3 board.

*Models are :*
- random AI
- minimax AI with max_depth = 9
- minimax AI with max_depth = 4
- minimax AI with max_depth = 1

*Warning :*
- It might take ~30 minutes to run all games and produce results (max_depth 9 is quite long to compute, especially if first to play)

NB : Each match is played twice with "odd/even" numbers swap

In [9]:
parameters = [('random','random',None),('mini1','minimax',1),('mini4','minimax',4),('mini9','minimax',9)] 

games = [Game(3,
              player0=AI_Player(name=i[0],algo=i[1],max_depth=i[2],verbose=False),
              player1 = AI_Player(name=j[0],algo=j[1],max_depth=j[2],verbose=False),
              verbose=False) 
              for i in parameters for j in parameters if i!=j]
for game in games:
    game.play()
    print(game.player0.name+" VS "+game.player1.name+" - Winner is : "+game.winner)

random VS mini1 - Winner is : Draw
random VS mini4 - Winner is : mini4
random VS mini9 - Winner is : mini9
mini1 VS random - Winner is : random
mini1 VS mini4 - Winner is : Draw
mini1 VS mini9 - Winner is : mini9
mini4 VS random - Winner is : mini4
mini4 VS mini1 - Winner is : mini4
mini4 VS mini9 - Winner is : Draw
mini9 VS random - Winner is : mini9
mini9 VS mini1 - Winner is : mini9
mini9 VS mini4 - Winner is : mini9


**Conclusion**

- minimax with depth 1 isn't really better than random
- minimax with depth 4 is better than random
- minimax with depth 9 is the best but it might take quite a long time to get the results